# Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [56]:
train_data_file = "SpamData/02_Training/train-data.txt"
test_data_file = "SpamData/02_Training/test-data.txt"

token_spam_prob_file = "SpamData/03_Testing/prob-spam.txt"
token_ham_prob_file = "SpamData/03_Testing/prob-nonspam.txt"
token_all_prob_file = "SpamData/03_Testing/prob-all-tokens.txt"

test_feature_matrix = "SpamData/03_Testing/test-features.txt"
test_target_file = "SpamData/03_Testing/test-target.txt"

vocab_size = 2500

# Read and Load Data

In [5]:
sparse_train_data = np.loadtxt(train_data_file, delimiter= ' ', dtype=int)

In [6]:
sparse_test_data = np.loadtxt(test_data_file, delimiter= ' ', dtype=int)

In [8]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 16,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [12]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in testing file', sparse_test_data.shape[0])
print('Nr of emails in traning data', np.unique(sparse_train_data[:, 0]).size)
print('Nr of emails in testing data', np.unique(sparse_test_data[:, 0]).size)

Nr of rows in training file 265555
Nr of rows in testing file 110579
Nr of emails in traning data 4014
Nr of emails in testing data 1723


Create Empty DataFrame

In [13]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, vocab_size))

In [14]:
index_names = np.unique(sparse_train_data[:, 0])

In [17]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create Full Matrix

In [18]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe
    Keyword Arguments:
    sparse_matrix: numpy array
    nr_words: size of the vocabulary. Total number of tokens.
    doc_idx: position of the document id in the sparse matrix. Default: 1st column
    word_idx: position of the word id in the sparse matrix. Default: 2nd column
    cat_idx: position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx: position of occurance of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, vocab_size))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix= pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)

    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurance = sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurance

    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [19]:
full_train_data = make_full_matrix(sparse_train_data, vocab_size)

In [20]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

Calculating the Probability of Spam

In [23]:
full_train_data.CATEGORY.size

4014

In [25]:
full_train_data.CATEGORY.sum()

1249

In [27]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Prob of spam', prob_spam)

Prob of spam 0.3111609367214748


Total nr of Words(Tokens)

In [28]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4014,)

In [30]:
total_wc = email_lengths.sum()
total_wc

444950

Nr of Tokens in Spam & Ham Emails

In [34]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1249,)

In [35]:
spam_wc = spam_lengths.sum()
spam_wc

193983

In [36]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [38]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0] # double check

0

In [37]:
ham_wc = ham_lengths.sum()
ham_wc

250967

In [40]:
spam_wc + ham_wc - total_wc # double check

0

Summing the tokens occuring in spam

In [41]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [42]:
train_spam_tokens.shape

(1249, 2500)

In [43]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1  #sum accors columns, we are adding 1 because we will do division in our probability calc. to make this calc nonzero we are doing Laplace Smoothing with adding 1

In [44]:
summed_spam_tokens.shape

(2500,)

In [45]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,0,0,0,0,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1898,0,3,1,0,2,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1899,0,1,0,0,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1900,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1901,2,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
train_ham_tokens.shape

(2765, 2500)

In [47]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [48]:
summed_ham_tokens.shape

(2500,)

### P(Token | Spam) 

In [49]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + vocab_size)   # we added vocab size because of Laplace Smooting
prob_tokens_spam[:5]

0    0.009064
1    0.004927
2    0.006881
3    0.010663
4    0.006917
dtype: float64

In [50]:
prob_tokens_spam.sum()

1.0

### P(Token | Ham) 

In [51]:
prob_tokens_nonspam = summed_ham_tokens / (ham_wc + vocab_size)
prob_tokens_nonspam.sum()

1.0

### P(Token)

In [52]:
prob_tokens_all = full_train_data.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0028070569726935

### Save the Trained Model

In [55]:
np.savetxt(token_spam_prob_file, prob_tokens_spam)
np.savetxt(token_ham_prob_file, prob_tokens_nonspam)
np.savetxt(token_all_prob_file, prob_tokens_all)

## Test Data

In [58]:
full_test_data = make_full_matrix(sparse_test_data, nr_words=vocab_size)

In [59]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [60]:
np.savetxt(test_target_file, y_test)
np.savetxt(test_feature_matrix, X_test)